# Execution Engine

Have questions? Chat with us on Github or Slack:

[![Homepage](https://img.shields.io/badge/fugue-source--code-red?logo=github)](https://github.com/fugue-project/fugue)
[![Slack Status](https://img.shields.io/badge/slack-join_chat-white.svg?logo=slack&style=social)](http://slack.fugue.ai)

We have already seen that the `transform()` function takes in an `engine`. There are three main ways to define the engine for Fugue.

## Setup

We can test all of the engines using this dummy setup. Note that the schema is already defined with a schema hint.

In [1]:
import pandas as pd
from fugue import transform 

df = pd.DataFrame({"col1": [1,2,3,4], "col2": [1,2,3,4]})

# schema: *, col3:int
def add_cols(df:pd.DataFrame) -> pd.DataFrame:
    return df.assign(col3 = df['col1'] + df['col2'])

ModuleNotFoundError: No module named 'pandas'

## Passing a String

This is the easiest to use. Normally, this means that the engine will be spun up locally and use all cores of the machine.

Fugue can take in the following strings: `"spark"`, `"dask"`, `"ray"`

**Spark**

In [7]:
spark_df = transform(df, add_cols, engine="spark")
spark_df.show()

+----+----+----+
|col1|col2|col3|
+----+----+----+
|   1|   1|   2|
|   2|   2|   4|
|   3|   3|   6|
|   4|   4|   8|
+----+----+----+



**Dask**

In [9]:
dask_df = transform(df, add_cols, engine="dask")
dask_df.compute().head()

,col1,col2,col3
0,1,1,2
0,2,2,4
0,3,3,6
0,4,4,8


**Ray**

In [17]:
ray_df = transform(df, add_cols, engine="ray")
ray_df.show(5)

Map_Batches: 100%|██████████| 1/1 [00:00<00:00, 68.18it/s]

{'col1': 1, 'col2': 1, 'col3': 2}
{'col1': 2, 'col2': 2, 'col3': 4}
{'col1': 3, 'col2': 3, 'col3': 6}
{'col1': 4, 'col2': 4, 'col3': 8}


## Passing the Client or Session

Fugue will also know how to interpret the engine if a Client or Session is passed.

**Spark**

In [19]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

spark_df = transform(df, add_cols, engine=spark)
spark_df.show()

+----+----+----+
|col1|col2|col3|
+----+----+----+
|   1|   1|   2|
|   2|   2|   4|
|   3|   3|   6|
|   4|   4|   8|
+----+----+----+



**Dask**

In [23]:
from distributed import Client
dask_client = Client()

dask_df = transform(df, add_cols, engine=dask_client)
dask_df.compute().head()

/opt/anaconda3/envs/fugue/lib/python3.8/site-packages/distributed/node.py:179: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 53898 instead
  warnings.warn(


,col1,col2,col3
0,1,1,2
0,2,2,4
0,3,3,6
0,4,4,8


**Ray**

This one is a bit different because Ray code doesn't invoke the Client.

In [26]:
import ray
ray.init(ignore_reinit_error=True)

ray_df = transform(df, add_cols, engine="ray")
ray_df.show(5)

Map_Batches: 100%|██████████| 1/1 [00:00<00:00, 64.40it/s]

{'col1': 1, 'col2': 1, 'col3': 2}
{'col1': 2, 'col2': 2, 'col3': 4}
{'col1': 3, 'col2': 3, 'col3': 6}
{'col1': 4, 'col2': 4, 'col3': 8}


## Passing the Client or Session

Fugue also has utilities to interact with clusters directly. For these, you need to be authenticated with the service like Databricks, Coiled or Anyscale.

The more complete documentation can be found in the [cloudprovider section](../integrations/cloudproviders/index.md)

Some examples:

```python
# Databricks
transform(tdf, dummy, engine="db", engine_conf=conf)

# Coiled
transform(df, add_cols, engine="coiled:my_cluster")

# Anyscale
transform(df, add_cols, engine="anyscale://project/cluster-1")
```

These will naturally require more configuration so the Fugue Cloudprovider documentation will be have more details.

## Engine Conf

As seen in the Databricks example above, we can also pass in a configuration to our execution engines. The most common one is:

In [27]:
spark_df = transform(df, 
                     add_cols, 
                     engine=spark, 
                     engine_conf={"fugue.spark.use_pandas_udf":True})
spark_df.show(2)

+----+----+----+
|col1|col2|col3|
+----+----+----+
|   1|   1|   2|
|   2|   2|   4|
+----+----+----+
only showing top 2 rows



For a full list of configurations, check [this page](https://fugue-tutorials.readthedocs.io/tutorials/advanced/useful_config.html#fugue-configurations)